In [1]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize data to the range [0, 1]
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Convert labels to one-hot encoded form
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [4]:
def build_model(hp):
    model = keras.Sequential()

    # Tune the number of convolutional filters in the first layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))

    # Second Conv2D Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu'
    ))

    # Pooling and Dropout
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

    # Third Conv2D Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values=[3, 5]),
        activation='relu'
    ))

    # Pooling and Dropout
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten and Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('dense_units', min_value=128, max_value=512, step=32), activation='relu'))
    model.add(layers.Dropout(rate=hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output layer
    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [5]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='cifar10_tuning',
    project_name='cifar10_optimization'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [7]:
tuner.search(x_train, y_train,
             epochs=20,
             validation_split=0.2,
             callbacks=[early_stopping])

Trial 30 Complete [00h 03m 20s]
val_accuracy: 0.7554000020027161

Best val_accuracy So Far: 0.777999997138977
Total elapsed time: 00h 31m 20s


In [9]:
# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Best number of filters for Conv2D layers:
- Layer 1: {best_hps.get('conv_1_filter')}
- Layer 2: {best_hps.get('conv_2_filter')}
- Layer 3: {best_hps.get('conv_3_filter')}
Best kernel sizes:
- Layer 1: {best_hps.get('conv_1_kernel')}
- Layer 2: {best_hps.get('conv_2_kernel')}
- Layer 3: {best_hps.get('conv_3_kernel')}
Best dropout rates:
- Dropout 1: {best_hps.get('dropout_1')}
- Dropout 2: {best_hps.get('dropout_2')}
- Dropout 3: {best_hps.get('dropout_3')}
Best dense layer units: {best_hps.get('dense_units')}
Best learning rate: {best_hps.get('learning_rate')}
""")


Best number of filters for Conv2D layers:
- Layer 1: 112
- Layer 2: 64
- Layer 3: 128
Best kernel sizes:
- Layer 1: 3
- Layer 2: 3
- Layer 3: 5
Best dropout rates:
- Dropout 1: 0.30000000000000004
- Dropout 2: 0.30000000000000004
- Dropout 3: 0.30000000000000004
Best dense layer units: 224
Best learning rate: 0.0006078879611407463



In [10]:
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
history = best_model.fit(x_train, y_train,
                         epochs=20,
                         validation_split=0.2,
                         callbacks=[early_stopping])

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.3274 - loss: 1.8115 - val_accuracy: 0.5922 - val_loss: 1.1599
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.5806 - loss: 1.1816 - val_accuracy: 0.6488 - val_loss: 1.0115
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.6599 - loss: 0.9732 - val_accuracy: 0.7008 - val_loss: 0.8732
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6984 - loss: 0.8640 - val_accuracy: 0.7254 - val_loss: 0.7867
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7253 - loss: 0.7840 - val_accuracy: 0.7357 - val_loss: 0.7670
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7457 - loss: 0.7156 - val_accuracy: 0.7544 - val_loss: 0.7126
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7678 - loss: 0.6620 - val_accuracy: 0.7586 - val_loss: 0.7054
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7840 - loss: 0.61

In [11]:
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7665 - loss: 0.7087
Test accuracy: 0.7556999921798706
